# Dynamical Systems: An Introduction

*(Adapted from Neuromatch Academy 2020, Week 2, Day 2, Tutorial 1)* https://colab.research.google.com/github/NeuromatchAcademy/course-content/blob/master/tutorials/W2D2_LinearSystems/student/W2D2_Tutorial1.ipynb

*Adriano Borsa, Mattia Rigotti --- CNTP March Workshop*

## Definition

A **dynamical system** is a system which evolves in time, for which the rules defining this time evolution can be described using a differential equation:

$\dot{x} = f(x)$,  

where $x$ is the state of the system and $\dot{x}$ is short-hand notation for $\frac{dx}{dt}$, that is, the derivative of $x$ with respect to time. The above equation then defines the rate of change of the system state as a function of the current state. In other words, given a certain state $x$, we can compute the direction in which this state will evolve across time.

**Note**: It is also possible for the dynamical system to be parametrized by time ($t$) or external inputs ($u$), in which case we would write:

$\dot{x} = f(x,t,u)$.

But, for today, we will focus on **autonomous/time-invariant** dynamical systems whose dynamics depend only on the current state.

## One-Dimensional Linear Differential Equations

Some of the simplest dynamical systems are described by an equation of the form:

$\dot{x} = a x$

where $a$ is a scalar constant and $x$ is a scalar variable. The solution for the above differential equation, which will give us an expression for how $x$ evolves in time, can be obtained:

$\int \frac{1}{x} \frac{dx}{dt} \, dt = \int a \, dt $ 

$\Rightarrow \log x = at + C$

$\Rightarrow x(t) = x_0\exp(at)$

where $x_0$ is the **intial condition** of the equation, that is, the value of $x$ at $t=0$. 

Let's plot this solution over time, for different values of $a$ and $x_0$.

In [ ]:
#@title Import dependencies
#@markdown Execute this cell
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp  # numerical integration solver

import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [ ]:
#@title Widget to Plot 1-D Dynamical System Solutions
#@markdown Execute to produce new widget
#@markdown Drag sliders to draw new trajectory

@widgets.interact(
    x0 = widgets.FloatSlider(value=0, min=-1, max=1, description="IC"),
    a = widgets.FloatSlider(value=0, min=-2, max=2, description="a"),
)
def plot_1D(x0, a):
  T = 2      # total Time duration
  dt = 0.001  # timestep of our simulation
  t = np.arange(0, T, dt) # time vector

  x = x0*np.exp(a*t)

  fig = plt.figure(figsize=(8,6))
  ax = fig.add_subplot(111)
  ax.set_ylim([-3,3])
  ax.set_xlabel("t")
  ax.set_ylabel("x")
  ax.set_title("Trajectory of the 1D Dynamical System.")
  ax.plot(t,x)

  fig.canvas.draw()


## Forward Euler Integration

In practice, differential equations are usually much more complex than the one-dimensional linear example above, and finding the analytical solution of $x(t)$ is not as straightforward. A common approach is to compute the solution to the equation numerically. To do so, we model time as a series of time steps $t_0, t_1, t_2, ...$, such that $t_{i+1} = t_i + dt$. Over a small time interval $dt$, we can get the corresponding change $dx$ from the definition of the differential:

$ \dot{x} = \frac{dx}{dt} \Rightarrow dx = \dot{x} \, dt$

So, at each time step  $t_i$ , we compute approximate the value of $x(t_i)$ as the sum of the value of  $x$  at the previous time step,  $x(t_{i−1})$,  and the small change over the timestep  $dx=\dot x dt$:


$$ x(t_i) = x(t_{i-1}) + \dot x(t_{i-1}) dt. $$

This very simple integration scheme, known as **forward Euler integration**, works well if  $dt$  is small and the ordinary differential equation is simple. It can run into issues when the ordinary differential equation is very noisy or when the dynamics include sudden big changes of  $x$. Such big jumps can occur, for example, in models of excitable neurons. In such cases, one needs to choose an integration scheme carefully. Note that differential equation solvers like *ode45* in MATLAB and *solve_ivp* in Python's scipy package use numerical methods similar in spirit to forward Euler integration, but which compute a higher order approximation of $dx$ (they evaluate $\dot x$ at several points, not just once at $t_{i−1}$ as we did here).

# The Pendulum: A Simple Dynamical System in Two Dimensions

## Description

The motion of a pendulum is a nice dynamical system to study since its behavior is intuitive. The parameters related to a pendulum's movement are shown in Figure 1. A point mass $m$ is connected by a rigid rod of length $L$ to a fixed point $P$. The point mass is subject to a gravitational constant of acceleration $g$. Finally, the angle of the pendulum is given by $\theta$.

<img src="https://www.acs.psu.edu/drussell/Demos/Pendulum/Pendulum.gif" style="width:100%"><figcaption align="left"> Figure 1 - Pendulum System. </figcaption>

The motion of the pendulum comes from physics first principles and can be described by the following dynamical system:

$$ \begin{array} \dot \dot x_1 = x_2 \\ \dot x_2 = -\frac{g}{L} \sin(x_1),
\end{array}$$

where the state variables $x_1$ and $x_2$ correspond to $\theta$ and $\dot \theta$ respectively. Let's investigate these equations a bit further. The first equation $\dot x_1 = x_2$ is just equivalent to saying $\dot \theta = \dot \theta$. This is always true regardless of the rules which govern the movement of the pendulum! The second equation is equivalent to writing $\ddot \theta = -\frac{g}{L} \sin(\theta)$. This is where the physics of the system come into play. Notice how the dynamical system we wrote actually describes a second-order differential equation!

## Simulation

We have a dynamical system in hand, so we're probably interested in what the trajectories look like. How does the pendulum's angular position ($x_1 = \theta$) and angular velocity ($x_2 = \dot \theta$) evolve through time if we were to initialize the system with certain initial conditions? A cooler way of saying the same thing is how does our dynamical system evolve through its **state space**? In this case, the initial conditions would be the initial angle of the pendulum (denoted $x_1(0)$) and the initial angular velocity of the pendulum (denoted $x_2(0)$). Once we start the system with some initial conditions, the rules of physics take over and the pendulum updates based on our dynamical equation.

### Quiver Plots

Quiver plots are useful tools for understanding **time-invariant** systems like that of the pendulum. Time-invariance just means that the dynamical system's update equations do not depend on time (except in the derivative). We only need to know the pendulum's instantaneous angle and velocity, which are stored in the state variables $x_1$ and $x_2$, to know how these quantities are changing over time. Quiver plots represent this information by plotting vectors in state space which represent the derivative of the state variables.

Let's see what this actually looks like for our pendulum example.

In [ ]:
#@title Import Dependencies
#@markdown Execute this cell
from matplotlib import pyplot as plt
from numpy.random import rand
import numpy as np
from scipy.integrate import odeint

import ipywidgets as widgets
from IPython.display import display
%matplotlib inline

In [ ]:
# parameters for quiver plot visualization
xmin=-5
xmax=5
ymin=-5
ymax=5

In [ ]:
# function that returns the derivative of a pendulum's dynamical system 
# at a given point x, t
def f_pend(x,t):

  # state variables (x passed in as a vector)
  x1 = x[0]
  x2 = x[1]

  # dynamics
  g=1 # just set to unity
  l=1 # just set to unity
  dx1dt=x2
  dx2dt=-(g/l)*np.sin(x1)
  
  # return output as a vector
  return np.array([dx1dt,dx2dt])

In [ ]:
#@title Plot Quiver
#@markdown Execute this cell
fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])
ax.set_title("Quiver Plot of Pendulum Dynamical System")
ax.set_xlabel("x_1")
ax.set_ylabel("x_2")

y1 = np.linspace(xmin, xmax, 20)
y2 = np.linspace(ymin, ymax, 20)
Y1, Y2 = np.meshgrid(y1, y2)
u, v = np.zeros(Y1.shape), np.zeros(Y2.shape)
NI, NJ = Y1.shape
for i in range(NI):
  for j in range(NJ):
    x = Y1[i, j]
    y = Y2[i, j]
    yprime = f_pend([x, y], 0)
    u[i,j] = yprime[0]
    v[i,j] = yprime[1]
ax.quiver(Y1, Y2, u, v, color='gray')
plt.show()

### Plotting Trajectories

Quiver plots give some idea of how the dynamical system updates, but we would like to see individual trajectories corresponding to specific initial conditions. Using **forward Euler Integration** or other numerical methods, we can solve forward from an initial condition to get these trajectories. Plotting individual trajectories over a quiver plot gives a nice visualization of the behavior of a dynamical system that can help build intuition.

In [ ]:
# forward Euler solver

# inputs: f is a function which describes the dynamical system
#       : ic is an np.array of the initial condition
#       : t is the time vector over which the simulation should occur...
#       note that the first element of t is the initial time

# outputs: x is the numerical solution to the simulation
def euler_solver(f, ic, t):

  # initialize array to store simulation solution
  x = np.zeros((len(t), len(ic)))
  x[0,:] = ic

  # iterate over simulation steps
  for ii in range(1, len(t)):

    # calculate derivative from previous time step
    x_dot = f(x[ii-1,:], t[ii-1])

    # calculate current time step & add to array
    dx = x_dot * (t[ii]-t[ii-1])
    x[ii,:] = x[ii-1,:] + dx
  
  return x

In [ ]:
# parameters for quiver plot visualization
xmin=-5
xmax=5
ymin=-5
ymax=5

# parameters for simulation
T = 20      # total Time duration
dt = 0.005  # timestep of our simulation
t = np.arange(0, T, dt) # time vector

In [ ]:
#@title Trajectory Plotting Widget
#@markdown Run this section to visualize trajectories. Use the sliders to change the initial conditions.

#@markdown Hit "Run Interact" to generate its corresponding trajectory. Run the section again to clear the output.

#@markdown Are the trajectories for Python's solver and your solver the same?

#@markdown What happens if you vary the value of dt?

# global variable to store trajectories
trajectories_package = []
trajectories_euler = []

def plot_trajectory(x_init, y_init, f, t, xmin, xmax, ymin, ymax):
  # update new trajectory, append to global variable
  solution=odeint(f,[x_init, y_init],t)
  solution_euler=euler_solver(f, np.array([x_init, y_init]),t)

  global trajectories_package
  global trajectories_euler
  trajectories_package.append(solution)
  trajectories_euler.append(solution_euler)
  
  # replot all solutions
  fig = plt.figure(figsize=(16,6))
  ax1 = fig.add_subplot(121)
  ax2 = fig.add_subplot(122)
  ax1.set_xlim([xmin,xmax])
  ax1.set_ylim([ymin,ymax])
  ax1.set_xlabel("x_1")
  ax1.set_ylabel("x_2")
  ax1.set_title("Python Numerical Solver")
  ax2.set_xlim([xmin,xmax])
  ax2.set_ylim([ymin,ymax])
  ax2.set_xlabel("x_1")
  ax2.set_ylabel("x_2")
  ax2.set_title("Forward Euler Implementation")

  # plot quiver
  y1 = np.linspace(xmin, xmax, 20)
  y2 = np.linspace(ymin, ymax, 20)
  Y1, Y2 = np.meshgrid(y1, y2)
  u, v = np.zeros(Y1.shape), np.zeros(Y2.shape)
  NI, NJ = Y1.shape
  for i in range(NI):
    for j in range(NJ):
      x = Y1[i, j]
      y = Y2[i, j]
      yprime = f_pend([x, y], 0)
      u[i,j] = yprime[0]
      v[i,j] = yprime[1]
  ax1.quiver(Y1, Y2, u, v, color='gray')
  ax2.quiver(Y1, Y2, u, v, color='gray')

  # plot all trajectories
  for trajectory in trajectories_package:
    ax1.plot(trajectory[:,0],trajectory[:,1],'b')
    ax1.plot(trajectory[0,0],trajectory[0,1],'r.')
  for trajectory in trajectories_euler:
    ax2.plot(trajectory[:,0],trajectory[:,1],'b')
    ax2.plot(trajectory[0,0],trajectory[0,1],'r.')
  fig.canvas.draw()

# interactive widget that plots new trajectories on a quiver plot
@widgets.interact_manual(
    x_1 = widgets.FloatSlider(value=np.floor((xmin+xmax)/2), min=xmin, max=xmax, description="x_1 init"),
    x_2 = widgets.FloatSlider(value=np.floor((ymin+ymax)/2), min=ymin, max=ymax, description="x_2 init"),
)
def plot(x_1, x_2):
  plot_trajectory(x_1, x_2, f_pend, t, xmin, xmax, ymin, ymax)

## A Slight Variation: Adding Friction

So far, we were considering the pendulum in the absence of friction. If we add friction to the system, it is described by the following dynamical system with an extra frictional term:

$$ \begin{array} \dot \dot x_1 = x_2 \\ \dot x_2 = -\frac{k}{m} x_2 -\frac{g}{L} \sin(x_1),
\end{array}$$

where $k$ is a coefficient of friction. Intuitively, we would expect that pendulum's motion would eventually stabilize at the origin (no angular velocity and the angular position being zero). Let's see what the dynamics say.

In [ ]:
def f_pend_fric(x,t):

  # state variables
  x1 = x[0]
  x2 = x[1]

  # dynamics
  m=1
  k=0.5
  g=1
  l=1
  dx1dt=x2
  dx2dt= -(k/m)*x2 - (g/l)*np.sin(x1)
  
  return np.array([dx1dt,dx2dt])

In [ ]:
# parameters for quiver plot visualization
xmin=-5
xmax=5
ymin=-5
ymax=5

# parameters for simulation
T = 20      # total Time duration
dt = 0.005  # timestep of our simulation
t = np.arange(0, T, dt) # time vector

In [ ]:
#@title Trajectory Plotting Widget
#@markdown Run this section to visualize trajectories. Use the sliders to change the initial conditions.

#@markdown Hit "Run Interact" to generate its corresponding trajectory. Run the section again to clear the output.

#@markdown To minimize ambiguity, each trajectory is plotted alone this time.

def plot_trajectory(x_init, y_init, f, t, xmin, xmax, ymin, ymax):
  # update new trajectory, append to global variable
  solution=odeint(f,[x_init, y_init],t)
  solution_euler=euler_solver(f, np.array([x_init, y_init]),t)
  
  # replot all solutions
  fig = plt.figure(figsize=(16,6))
  ax1 = fig.add_subplot(121)
  ax2 = fig.add_subplot(122)
  ax1.set_xlim([xmin,xmax])
  ax1.set_ylim([ymin,ymax])
  ax1.set_xlabel("x_1")
  ax1.set_ylabel("x_2")
  ax1.set_title("Python Numerical Solver")
  ax2.set_xlim([xmin,xmax])
  ax2.set_ylim([ymin,ymax])
  ax2.set_xlabel("x_1")
  ax2.set_ylabel("x_2")
  ax2.set_title("Forward Euler Implementation")

  # plot quiver
  y1 = np.linspace(xmin, xmax, 20)
  y2 = np.linspace(ymin, ymax, 20)
  Y1, Y2 = np.meshgrid(y1, y2)
  u, v = np.zeros(Y1.shape), np.zeros(Y2.shape)
  NI, NJ = Y1.shape
  for i in range(NI):
    for j in range(NJ):
      x = Y1[i, j]
      y = Y2[i, j]
      yprime = f_pend_fric([x, y], 0)
      u[i,j] = yprime[0]
      v[i,j] = yprime[1]
  ax1.quiver(Y1, Y2, u, v, color='gray')
  ax2.quiver(Y1, Y2, u, v, color='gray')

  # plot all trajectories
  ax1.plot(solution[:,0],solution[:,1],'b')
  ax1.plot(solution[0,0],solution[0,1],'r.')

  ax2.plot(solution_euler[:,0],solution_euler[:,1],'b')
  ax2.plot(solution_euler[0,0],solution_euler[0,1],'r.')
  fig.canvas.draw()


# interactive widget that plots new trajectories on a quiver plot
@widgets.interact_manual(
    x_1 = widgets.FloatSlider(value=np.floor((xmin+xmax)/2), min=xmin, max=xmax, description="x_1 init"),
    x_2 = widgets.FloatSlider(value=np.floor((ymin+ymax)/2), min=ymin, max=ymax, description="x_2 init"),
)
def plot(x_1, x_2):
  plot_trajectory(x_1, x_2, f_pend_fric, t, xmin, xmax, ymin, ymax)

# Dynamical Systems in Computational Neuroscience

## Computation Through Neural Population Dynamics

How do these concepts of dynamical systems apply to neuroscience? 

The Computation Through Dynamics framework states that though the activity of individual neurons is subject to variability, the activity of populations of neurons is more consistent, and evolves across time according to internal dynamics. The dynamics of a latent state $z(t)$ (usually of lower dimension than the number of neurons) are distributed across the firing rates $x(t)$ of a population of neurons to carry out computations which result in a certain output behavior. The dynamics of the latent state are described as follows:

$\dot{\mathbf{z}} = f(\mathbf{z},\mathbf{u})$

Where $u(t)$ are external inputs to the system (e.g. neuron projections from other brain areas) which influence the time evolution of the state. For simplicity of this demo, we are going to ignore the effect of these external inputs, and model the dynamics of the latent state as:

$\dot{\mathbf{z}} = f(\mathbf{z})$

The above equation corresponds to **autonomous dynamics**, that is, a dynamical system which does not depend on external inputs, so the time evolution of the state can be entirely predicted from its initial conditions. Though this behavior has been observed in some brain areas during certain types of tasks, it is not usually the case. 

In turn, the relation between the latent state and individual firing rates is given as follows:

$\mathbf{x}(t) = W \mathbf{z}(t)$

Where $W$ is a readout matrix that maps the individual firing rate for all neurons from the population latent state $z(t)$.

We will now show a brief demo of how this would look in practice, for a neuron population exhibiting autonomous dynamics.

## Model

We define the latent dynamics which governs the activity of a specific population of neurons as follows:

$ \dot{\mathbf{z}} = \begin{bmatrix} 0 & 2 \\ -2 & 0
\end{bmatrix}\begin{bmatrix} z_1 \\ z_2
\end{bmatrix} + \text{offset} $

Where the offset term is added such that the latent state has positive values (so it doesn't map onto negative firing rates). Note the similarities of the above matrix with the dynamical system we defined earlier for the pendulum.

As mentioned before, the dimension of the latent state $\mathbf{z}(t)$ is usually lower than the number of neurons in the population. In this case, we will have $N=3$ neurons in the population. Let's map the relation between the latent state $\mathbf{z}(t)$ and resulting firing rate of each neuron $\mathbf{x}(t)$ as follows:

$\mathbf{x} = \begin{bmatrix} x_1 \\ x_2 \\ x_3
\end{bmatrix} = \begin{bmatrix} 1 & 0 \\ 0 & 1 \\ 1 & 1
\end{bmatrix}\begin{bmatrix} z_1 \\ z_2
\end{bmatrix}$

And lastly, we model individual neuron spiking as a Poisson process with rate $x(t)$ at a given time. That is, for a given sample $t$ and bin size $T$, the number of spikes $N$ in that given time bin is defined by the following probability distribution:

$P[N=n] = \frac{(x(t) T)^n}{n!} e^{-x(t) T}$

In [ ]:
# Define dynamics for latent state
def f(z):

    dz1dt=2*z[1] - 8
    dz2dt=-2*z[0] + 8
    
    dzdt = np.array([dz1dt,dz2dt])

    return dzdt


# Define readout matrix
W = np.array([[1, 0],[0, 1],[1, 1]])


# Generate spike trains from firing rates, for severals trials
def generate_spikes(x,dt,trials):
  
  x_N = np.tile(x, (trials ,1, 1))
  spikes = np.random.poisson(x_N*dt)
  
  return spikes

Let's assume that different task conditions (e.g. reaching with your arm to the left v/s reaching to the right) are represented by specific initial conditions of the latent state. Let's simulate the resulting population spikes for a certain initial condition:

In [ ]:
#@title Population Spiking for IC1
#@markdown Execute this cell

dt = 0.001
t = np.arange(0,1,dt)

# initial condition 1
z1_0 = np.array([[2, 4]])

# Euler integration to generate latent state trayectory
z1 = np.zeros((2,t.shape[0]))
for i, ti in enumerate(t):
  if i == 0:
    z1[:,i] = z1_0
  else:
    z1[:,i] = z1[:,i-1] + dt*f(z1[:,i-1])

# Map latent state to neuron firing rates
x1 = np.dot(W,z1)

# Seed for random data
np.random.seed(2021)

trials = 10

# Generate spikes
spikes1 = generate_spikes(x1,dt,trials)

fig, axs = plt.subplots(1, 3, figsize=(15,6))
for n in range(W.shape[0]):
  for t in range(trials):
    spike_times = np.nonzero(spikes1[t,n])
    axs[n].eventplot(spike_times[0]*dt, lineoffsets=t)
    axs[n].set_xlabel('Time [s]')
    axs[n].set_ylabel('Trial #')
    axs[n].set_title('Neuron {}'.format(n+1))
fig.suptitle('Population spike trains for first initial condition')
plt.show()


Note that there is significant variability from trial to trial. Moreover, it is hard to make out the underlying fire rates (this is why it is common procedure to average and smooth the spike trains among several trials). 

Repeating for a different initial condition in the latent state:

In [ ]:
#@title Population Spiking for IC2
#@markdown Execute this cell

dt = 0.001
t = np.arange(0,1,dt)

# initial condition 2
z2_0 = np.array([[6, 4]])

# Euler integration to generate latent state trayectory
z2 = np.zeros((2,t.shape[0]))
for i, ti in enumerate(t):
  if i == 0:
    z2[:,i] = z2_0
  else:
    z2[:,i] = z2[:,i-1] + dt*f(z2[:,i-1])

# Map latent state to neuron firing rates
x2 = np.dot(W,z2)

# Seed for random data
np.random.seed(2021)

trials = 10

# Generate spikes
spikes2 = generate_spikes(x2,dt,trials)

fig, axs = plt.subplots(1, 3, figsize=(15,6))
for n in range(W.shape[0]):
  for t in range(trials):
    spike_times = np.nonzero(spikes2[t,n])
    axs[n].eventplot(spike_times[0]*dt, lineoffsets=t)
    axs[n].set_xlabel('Time [s]')
    axs[n].set_ylabel('Trial #')
    axs[n].set_title('Neuron {}'.format(n+1))
fig.suptitle('Population spike trains for second initial condition')
plt.show()

Again, there is a lot of variability among trials, and it is hard to uncover the underlying firing rates. We do see some differences in the overall firing frequencies of neurons 1 and 2 for this condition compared to previous one, but that's about it. 

What if we analyzed the activity of this neuron population through the underlying dynamics which generated these spike trains?

In [ ]:
#@title Plot Quiver and Trajectories
xmin=0
xmax=8
ymin=0
ymax=8


fig = plt.figure(figsize=(8,6))
ax = fig.add_subplot(111)
ax.set_xlim([xmin,xmax])
ax.set_ylim([ymin,ymax])
ax.set_title("Trajectory of latent state for both conditions")
ax.set_xlabel("z_1")
ax.set_ylabel("z_2")

y1 = np.linspace(xmin, xmax, 20)
y2 = np.linspace(ymin, ymax, 20)
Y1, Y2 = np.meshgrid(y1, y2)
u, v = np.zeros(Y1.shape), np.zeros(Y2.shape)
NI, NJ = Y1.shape
for i in range(NI):
  for j in range(NJ):
    x = Y1[i, j]
    y = Y2[i, j]
    z = np.array([x,y])
    yprime = f([x, y])
    u[i,j] = yprime[0]
    v[i,j] = yprime[1]
ax.quiver(Y1, Y2, u, v, color='gray')

plt.plot(z1[0,0],z1[1,0],'bo')
plt.plot(z1[0,:],z1[1,:],'b')
plt.plot(z1[0,-1],z1[1,-1],'bx')

plt.plot(z2[0,0],z2[1,0],'ro')
plt.plot(z2[0,:],z2[1,:],'r')
plt.plot(z2[0,-1],z2[1,-1],'rx')

plt.show()

If we directly look at evolution of the latent state, we observe structured trajectories that are clearly distinguishable for both conditions, which was not the case when observing the individual neuron spikes. This is expected, since we modeled the system as so. 

Also, since this neural population was modeled as an autonomous system, if we were able to uncover the dynamics and initial conditions of the latent state for a specific trial (which we assumed as known for this demo), the trajectory of the latent state, and thus population firing rates (which are modeled as a linear mapping from the latent state) could be predicted on a trial-by-trial basis! 

It is clear then that, when possible, there are several benefits to representing the activity of a population of neurons as a dynamical system. This framework allows us to represent the noisy activity of a population of neurons as a lower-dimensional, more constrained time-evolving state.

Some questions come to mind though:
*   How do we know if a dynamical system is a good model for the activity of specific population of neuron?
*   If the above is true, how do we uncover these neural population dynamics?
*   How do we obtain the initial condition of the latent state in practice?
*   What if communication with other brain areas is relevant, how do we model the external inputs $u(t)$?

Current research on this topic focuses on the above questions. For further reading, we suggest the following review article which goes over some of these works: https://web.stanford.edu/~mgolub/publications/2020-Vyas-ARN.pdf